# Imports 

In [1]:
print('hi')

hi


In [2]:
%matplotlib inline
import os
import sys
import inspect
import numpy as np
from scipy.special import factorial
import h5py

#data_path = '/data'     ... data path specified later
#data_path
#initial_pulse = '../pulses/example_pulses/transmon_cat_initial_pulse.h5'
from h5py import File
import matplotlib.pyplot as plt
from pylab import*
from qutip import*

from scipy import interpolate
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)


In [3]:
sys.path.append(r'C:\_Lib\python\quantum-optimal-control')

In [4]:
from quantum_optimal_control.helper_functions.grape_functions import *
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control import*

In [5]:
from IPython.display import clear_output

In [6]:
%load_ext autoreload
%autoreload 2

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

# 3DMM2 parameters

In [7]:
# chi = Peak splitting MHz
chis = array([-33, -33]) 
kappas  = array([0,0]) # kHz

# Multimode circle grape

In [8]:
#TODO: Figure out why relative import doesn't work
sys.path.append(r'C:\_Lib\python\Multimode-Conditional-Displacements\hpc_runs\multimode_circle_grape\class_description')
from circle_grape_v4 import *

done


In [9]:
transmon_levels = 2

# Data Storage

In [10]:
import pandas as pd
fname = 'dataset_sweep_qdrive_detuning2.csv'
columns=['time', 'steps','alpha', 'detuning', 'qubit_drive_amp','err', 'filenum']#total_time, steps, alpha, detuning,  qubit_drive_amp, err, filenum
df = None

# Main

In [11]:
qubit_drives = [0.005, 0.01, 0.015, 0.02, 0.03 ] 
detunings = [0.005, 0.01, 0.015, 0.02, 0.03 ] #Ghz

In [12]:
states_forbidden_list = []

convergence = {'rate': 0.1, 'update_step': 20, 'max_iterations': 250,
               'conv_target': 1e-3, 'learning_rate_decay': 500.0}




initial_guess = None

In [13]:
data_path ='' #'C:/Users/Chakram_Meas/Documents/GitHub/Multimode-Conditional-Displacements/hpc_runs/multimode_circle_grape/Grape on multiple modes/Data'

In [16]:
def main(fname = fname, df = df):
    '''
    Vary time length, get fidelity
    '''
    filenum = 10

    for drive_amp in qubit_drives:
        for detuning in detunings:
            #cuz data already got for certain points
            if drive_amp ==0.005 and (detuning in [0.005, 0.01, 0.015, 0.02]):
                continue
            else:
                clear_output(wait = True)
                total_time = 1000
                steps = 1000
                alpha = 30
                #param

                mode = 2
                mode_levels = 10
                chi,kappa = chis[:mode],kappas[:mode]
                chi = [i*1e-6 for i in chi]
                kappa = [i*1e-6 for i in kappa]
                circle_grape_params = {"chis":chi,"kappas":kappa,"alpha":alpha,"delta_c":detuning}


                reg_coeffs = {'dwdt': 0.1, 'd2wdt2': 1.0e-3, 'forbid_dressed': False,
                              'states_forbidden_list':states_forbidden_list,
                              'forbidden_coeff_list': [1.0*steps] * len(states_forbidden_list)}

                op = multimode_circle_grape_optimal_control(mode_state_num = mode_levels,
                                                            number_of_modes = mode,hparams = circle_grape_params,add_disp_kerr=False)



                qubit_drive_amp = drive_amp # Ghz
                ss = op.run_optimal_control(state_transfer = True, initial_states = [5], target_states = [mode_levels + 5], 
                                total_time = total_time, steps = steps,max_amp = qubit_drive_amp, 
                                taylor_terms = None,is_dressed=False, 
                                convergence = convergence, reg_coeffs =  reg_coeffs,
                                plot_only_g = True,
                                states_forbidden_list = states_forbidden_list,initial_guess = initial_guess, 
                                file_name="g01_to_g10_circlgrape" + str(filenum), data_path=data_path, save = True)
                hf = op.openfile()
                err  = min(hf['error'])
                if np.isnan(err):
                    err = -1
                df_new = pd.DataFrame([[total_time, steps, alpha, detuning,  qubit_drive_amp, err, filenum]], columns=columns)
                if df is None : 
                    df = df_new
                else:
                    df = df.append(df_new, ignore_index = True)
                df.to_csv(fname, index=False)
                filenum+=1

In [ ]:
main()

['H0', 'Hnames', 'Hops', 'U', 'convergence', 'error', 'initial_vectors_c', 'inter_vecs_imag', 'inter_vecs_mag_squared', 'inter_vecs_raw_imag', 'inter_vecs_raw_real', 'inter_vecs_real', 'iteration', 'maxA', 'method', 'reg_coeffs', 'reg_error', 'run_time', 'sparse_H', 'sparse_K', 'sparse_U', 'states_concerned_list', 'steps', 'taylor_scaling', 'taylor_terms', 'total_time', 'uks', 'unitary_scale', 'use_gpu']
0.9262096
